In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
os.chdir("D:\machine_learning\stellar_classification")
import time

In [3]:
device=torch.device("cpu")

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
#device=torch.device("cpu")
device

device(type='cpu')

In [5]:
def clscvrt(v):
    return np.where(cls==v)[0][0]

In [6]:
df = pd.read_csv("star_classification.csv")
cls=np.array(list(set(df["class"])))

df["class"]=df["class"].apply(clscvrt, args=())
cols=list(df.columns)
cols[-5]=df.columns[-1]
cols[-1]="class"
df=df[cols]
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,fiber_ID,redshift,plate,MJD,class
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,171,0.634794,5812,56354,0
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,427,0.779136,10445,58158,0
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,299,0.644195,4576,55592,0
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,775,0.932346,9149,58039,0
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,842,0.116123,6121,56187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,438,0.000000,9374,57749,0
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,866,0.404895,7626,56934,0
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,74,0.143366,2764,54535,0
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,470,0.455040,6751,56368,0


In [7]:
train, valid = np.split(df.sample(frac=1), [int(0.8*len(df))])
print(len(train),len(valid))

80000 20000


C:\Users\Kiran M R\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y

In [9]:
train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
data_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)

In [10]:
# # 10 hidden layer model

# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, dropout_prob):
#         super(NeuralNetwork, self).__init__()
        
#         # Define the layers
#         self.hidden1 = nn.Linear(input_size, hidden_size)
#         self.hidden2 = nn.Linear(hidden_size, hidden_size)
#         self.hidden3 = nn.Linear(hidden_size, hidden_size)
#         self.hidden4 = nn.Linear(hidden_size, hidden_size)
#         self.hidden5 = nn.Linear(hidden_size, hidden_size)
#         self.hidden6 = nn.Linear(hidden_size, hidden_size)
#         self.hidden7 = nn.Linear(hidden_size, hidden_size)
#         self.hidden8 = nn.Linear(hidden_size, hidden_size)
#         self.hidden9 = nn.Linear(hidden_size, hidden_size)
#         self.hidden10 = nn.Linear(hidden_size, hidden_size)
#         self.dropout = nn.Dropout(p=dropout_prob)
#         self.output = nn.Linear(hidden_size, output_size)
        
#         # Define activation function (e.g., ReLU)
#         self.activation = nn.ReLU()
    
#     def forward(self, x):
#         x = self.activation(self.hidden1(x))
#         x = self.activation(self.hidden2(x))
#         x = self.activation(self.hidden3(x))
#         x = self.activation(self.hidden4(x))
#         x = self.activation(self.hidden5(x))
#         x = self.activation(self.hidden6(x))
#         x = self.activation(self.hidden7(x))
#         x = self.activation(self.hidden8(x))
#         x = self.activation(self.hidden9(x))
#         x = self.activation(self.hidden10(x))
#         #x = torch.softmax(self.output_layer(x))
#         x = self.dropout(x)
#         x = self.output(x)
#         x=nn.Softmax(dim=1)(x)
#         return x

# # Define the input size, hidden size, and output size
# input_size = len(df.columns)-1  # Change this to match the dimension of your input data)
# hidden_size = 32  # Number of neurons in each hidden layer
# output_size = 3  # Number of output nodes

# # Create an instance of the neural network
# model = NeuralNetwork(input_size, hidden_size, output_size,0.0)

# # You can now use this model for training and making predictions.



In [ ]:
# # 10 hidden layer model

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, dropout_prob):
#         super(NeuralNetwork, self).__init__()
        
#         # Define the layers
#         self.hidden1 = nn.Linear(input_size, hidden_size)
#         self.hidden2 = nn.Linear(hidden_size, hidden_size)
#         self.hidden3 = nn.Linear(hidden_size, hidden_size)
#         self.hidden4 = nn.Linear(hidden_size, hidden_size)
#         self.hidden5 = nn.Linear(hidden_size, hidden_size)
#         # self.hidden6 = nn.Linear(hidden_size, hidden_size)
#         # self.hidden7 = nn.Linear(hidden_size, hidden_size)
#         # self.hidden8 = nn.Linear(hidden_size, hidden_size)
#         # self.hidden9 = nn.Linear(hidden_size, hidden_size)
#         # self.hidden10 = nn.Linear(hidden_size, hidden_size)
#         self.dropout = nn.Dropout(p=dropout_prob)
#         self.output = nn.Linear(hidden_size, output_size)
        
#         # Define activation function (e.g., ReLU)
#         self.activation = nn.ReLU()
    
#     def forward(self, x):
#         x = self.activation(self.hidden1(x))
#         x = self.activation(self.hidden2(x))
#         x = self.activation(self.hidden3(x))
#         x = self.activation(self.hidden4(x))
#         x = self.activation(self.hidden5(x))
#         # x = self.activation(self.hidden6(x))
#         # x = self.activation(self.hidden7(x))
#         # x = self.activation(self.hidden8(x))
#         # x = self.activation(self.hidden9(x))
#         # x = self.activation(self.hidden10(x))    
#         x = self.dropout(x)
#         x = self.output(x)
#         x = F.softmax(x, dim=1)  # Use nn.functional.softmax
#         return x



# # Define the input size, hidden size, and output size
# input_size = len(df.columns)-1  # Change this to match the dimension of your input data)
# hidden_size = 64  # Number of neurons in each hidden layer
# output_size = 3  # Number of output nodes

# # Create an instance of the neural network
# dropout_prob = 0.1
# model = NeuralNetwork(input_size, hidden_size, output_size,dropout_prob).to(device)
# loss_function = nn.CrossEntropyLoss()  # For classification, adjust for your specific problemx

# learning_rate = 0.01
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# one_hot_labels=np.eye(3)[y_train]
# features_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
# labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)
# c,m=0,0


In [11]:
one_hot_labels=np.eye(3)[y_train]
features_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)

In [12]:

from torch.utils.data import TensorDataset

dataset = TensorDataset(features_tensor, labels_tensor)

In [13]:
features_tensor

tensor([[-0.3965,  0.2924,  1.0957,  ...,  2.0102,  0.7766,  0.6371],
        [-0.2614,  0.5550,  0.9703,  ..., -0.7092, -1.1697, -1.1901],
        [-1.7000, -0.2057, -1.3028,  ..., -0.7866, -0.4566, -0.1734],
        ...,
        [-0.6379,  0.3237,  1.7551,  ..., -0.7892,  1.0371,  1.0173],
        [-0.1091, -1.5297, -1.1863,  ..., -0.7886, -0.7676, -0.6127],
        [ 1.6455, -1.7482,  0.0064,  ..., -0.7879, -0.6766, -0.4514]])

In [14]:
labels_tensor

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [15]:
print("Y_train\n",y_train,"\nOne_hot_labels\n",one_hot_labels)

Y_train
 [1 0 2 ... 2 2 2] 
One_hot_labels
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [16]:
# 2 hidden layer model

import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(NeuralNetwork, self).__init__()
        
        # Define the layers
        self.hidden1 = nn.Linear(input_size, hidden_size)
        self.hidden2 = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(p=dropout_prob)
        self.output = nn.Linear(hidden_size, output_size)
        
        # Define activation function (e.g., ReLU)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.hidden1(x))
        x = self.activation(self.hidden2(x))
    
        x = self.dropout(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)  # Use nn.functional.softmax.to(device)
        return x



# Define the input size, hidden size, and output size
input_size = len(df.columns)-1  # Change this to match the dimension of your input data)
hidden_size = 64  # Number of neurons in each hidden layer
output_size = 3  # Number of output nodes
dropout_prob = 0.19
d_p = dropout_prob

# Create an instance of the neural network
model = NeuralNetwork(input_size, hidden_size, output_size,dropout_prob).to(device)
loss_function = nn.CrossEntropyLoss()  # For classification, adjdropout_probt for your specific problemx

data_tensor=X_train
learning_rate= 0.008
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

one_hot_labels=np.eye(3)[y_train]
features_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)
c,m=0,0
tim = time.ctime()
tim=tim[11:19]+ '_'+ tim[20:]
name="model_data_" + tim.replace(':', '.') + ".txt"

with open(name, "a") as f:
    f.write("Dropout Probability = " + str(dropout_prob) + "\n")
    f.write("learning rate = " + str(learning_rate) + "\n")


In [17]:
tim = time.ctime()
tim=tim[11:19]+ '_'+ tim[20:]

In [18]:
from torch.utils.data import DataLoader

batch_size =1024  # Adjust the batch size as needed
shuffle = True   # Set to True if you want to shuffle the data during each epoch

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [19]:

# num_epochs = 450  # Adjust the number of training epochs as needed
# print("Multiplier:", c+1)
# for epoch in range(num_epochs-m):
    
#     model.train()  # Set the model to training mode

#     for batch_data, batch_labels in dataloader:  # dataloader is a DataLoader object with your training data
#         optimizer.zero_grad()  # Zero the gradients

#         # Forward pass
#         outputs = model(batch_data)
#         loss = loss_function(outputs, batch_labels)

#         # Backpropagation
#         loss.backward()
#         optimizer.step()

#     # Print or log the loss for this epoch
#     print(f'Epoch [{c*num_epochs+m+1}/{num_epochs*(c+1)}], Loss: {loss.item()}')

#     # After training, save the model or use it for predictions
#     torch.save(model.state_dict(), 'your_model.pth')
#     m+=1

# #print("Actual no. of epoch =",c*num_epochs+epoch+1)
# c+=1
# m=0



In [20]:
# data_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)
# chumma = 0
# Nt=10
# for i in range(Nt):
#     pred=model(data_tensor)
#     pred=pred.to('cpu')
#     pred=pred.detach().numpy()
#     prediction=pred.copy()
#     i=0
#     ind=np.arange(3)
#     index=[]
#     for a in prediction:
#         pred[i]=np.array([0,0,0])
#         #print(a)
#         max_ind=np.argmax(a)
#         index.append(max_ind)

#         #print(max_ind)
#         pred[i][max_ind]=1
#         #print(pred[i])
#         i+=1
        
#     count=0
#     for i in range(len(pred)):
#         if index[i] == y_valid[i]:
#             count+=1
#     chumma += count/len(pred)
#     print(count/len(pred))
# print(f'Average is (%) = {chumma/Nt*100} (epoch = {c*num_epochs+m})')
    

In [22]:
data_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)
num_epochs = 450  # Adjust the number of training epochs as needed
print("Multiplier:", c+1)

model.train()  # Set the model to training mode

for batch_data, batch_labels in dataloader:  # dataloader is a DataLoader object with your training data
    optimizer.zero_grad()  # Zero the gradients

    # Forward pass
    outputs = model(batch_data)
    loss = loss_function(outputs, batch_labels)

    # Backpropagation
    loss.backward()
    optimizer.step()
    now_loss = loss.item()

for epoch in range(1,num_epochs-m):

    model.train()  # Set the model to training mode

    for batch_data, batch_labels in dataloader:  # dataloader is a DataLoader object with your training data
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        outputs = model(batch_data)
        loss = loss_function(outputs, batch_labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        
    pre_loss = now_loss
    now_loss = loss.item()
    
    if now_loss > pre_loss:
        learning_rate = learning_rate * 0.999
        d_p = d_p * 1.005
    else:
        learning_rate = learning_rate * 1.0005
        d_p = d_p * 0.9995


    model.dropout.p = d_p
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    # model=new_model
    # new_model = NeuralNetwork(input_size, hidden_size, output_size,d_p_new).to(device)
    # # Copy the weights from the pre-trained model to the new model
    # new_model.load_state_dict(model.state_dict())

    # optimizer = torch.optim.SGD(new_model.parameters(), lr=lr_new)

    # Print or log the loss for this epoch
    curr_epo=c*num_epochs+m+1
    print(f'Epoch [{curr_epo}/{num_epochs*(c+1)}], Loss: {loss.item()}')

    # After training, save the model or use it for predictions
    torch.save(model.state_dict(), 'your_model.pth')


    if((curr_epo+1)%20<1):
        ##data_tensor = torch.tensor(X_valid, dtype=torch.float32).to(device)

        chumma = 0
        Nt=10
        for i in range(Nt):
            pred=model(data_tensor)
            pred=pred.to('cpu')
            pred=pred.detach().numpy()
            prediction=pred.copy()
            i=0
            ind=np.arange(3)
            index=[]
            for a in prediction:
                pred[i]=np.array([0,0,0])
                #print(a)
                max_ind=np.argmax(a)
                index.append(max_ind)

                #print(max_ind)
                pred[i][max_ind]=1
                #print(pred[i])
                i+=1
                
            count=0
            for i in range(len(pred)):
                if index[i] == y_valid[i]:
                    count+=1
            chumma += count/len(pred)
            #print(count/len(pred))
        print(f'Average is (%) = {chumma/Nt*100} (epoch = {c*num_epochs+m})')
        with open(name, "a") as f:
            f.write(f'Average accuracy is (%) = {chumma/Nt*100} (epoch = {c*num_epochs+m+1})' + "\n")

    m+=1

#print("Actual no. of epoch =",c*num_epochs+epoch+1)
c+=1
m=0



Multiplier: 1
Epoch [1/450], Loss: 1.0726308822631836
Epoch [2/450], Loss: 1.0536401271820068
Epoch [3/450], Loss: 1.0166748762130737
Epoch [4/450], Loss: 0.9990162253379822
Epoch [5/450], Loss: 1.0147935152053833
Epoch [6/450], Loss: 0.9844797253608704
Epoch [7/450], Loss: 0.9663738012313843
Epoch [8/450], Loss: 0.912004828453064
Epoch [9/450], Loss: 0.9518927335739136
Epoch [10/450], Loss: 0.9405859708786011
Epoch [11/450], Loss: 0.8954501748085022
Epoch [12/450], Loss: 0.9346120953559875
Epoch [13/450], Loss: 0.89791339635849
Epoch [14/450], Loss: 0.9211969971656799
Epoch [15/450], Loss: 0.8859829902648926
Epoch [16/450], Loss: 0.9072633385658264
Epoch [17/450], Loss: 0.9209749102592468
Epoch [18/450], Loss: 0.8855128288269043
Epoch [19/450], Loss: 0.8519555330276489
Epoch [20/450], Loss: 0.8750988841056824
Average is (%) = 49.164 (epoch = 19)
Epoch [21/450], Loss: 0.8344117403030396
Epoch [22/450], Loss: 0.8501617312431335
Epoch [23/450], Loss: 0.8681443929672241
Epoch [24/450], Lo

lr = 0.028
dropout_prob = 0.05

In [20]:
f.close()